In [3]:
include("../read.jl")
include("../constraints.jl")
include("../partitions.jl")
include("../problem.jl")
include("../active.jl")

jobs = build_jobs("/Users/jpoullet/Documents/MIT/Courses/15.094 Robust optimization/RobustProject/data/example.csv", [0, 0])
problem = Problem(jobs, 2, 10, 1e9)



INFO: Precompiling module CSV.


LoadError: LoadError: ArgumentError: Module LinearAlgebra not found in current path.
Run `Pkg.add("LinearAlgebra")` to install the LinearAlgebra package.
while loading /Users/jpoullet/Documents/MIT/Courses/15.094 Robust optimization/RobustProject/src/read.jl, in expression starting on line 3

In [2]:
Pkg.add("CSV")

INFO: Cloning cache of CSV from https://github.com/JuliaData/CSV.jl.git
INFO: Cloning cache of CategoricalArrays from https://github.com/JuliaData/CategoricalArrays.jl.git
INFO: Cloning cache of DataArrays from https://github.com/JuliaStats/DataArrays.jl.git
INFO: Cloning cache of DataFrames from https://github.com/JuliaData/DataFrames.jl.git
INFO: Cloning cache of DataStreams from https://github.com/JuliaData/DataStreams.jl.git
INFO: Cloning cache of DataStructures from https://github.com/JuliaCollections/DataStructures.jl.git
INFO: Cloning cache of FileIO from https://github.com/JuliaIO/FileIO.jl.git
INFO: Cloning cache of GZip from https://github.com/JuliaIO/GZip.jl.git
INFO: Cloning cache of NullableArrays from https://github.com/JuliaStats/NullableArrays.jl.git
INFO: Cloning cache of Reexport from https://github.com/simonster/Reexport.jl.git
INFO: Cloning cache of SortingAlgorithms from https://github.com/JuliaCollections/SortingAlgorithms.jl.git
INFO: Cloning cache of SpecialFunc

In [ ]:
Pkg.add("LinearAlgebra")

In [9]:
using JuMP, JuMPeR, Gurobi

INFO: Precompiling module JuMP.

Use "Dict(a=>b for (a,b) in c)" instead.
INFO: Precompiling module Gurobi.


In [7]:
Pkg.add("JuMP")


INFO: Cloning cache of Calculus from https://github.com/JuliaMath/Calculus.jl.git
INFO: Cloning cache of DiffBase from https://github.com/JuliaAttic/DiffBase.jl.git
INFO: Cloning cache of ForwardDiff from https://github.com/JuliaDiff/ForwardDiff.jl.git
INFO: Cloning cache of JuMP from https://github.com/JuliaOpt/JuMP.jl.git
INFO: Cloning cache of MathProgBase from https://github.com/JuliaOpt/MathProgBase.jl.git
INFO: Cloning cache of NaNMath from https://github.com/mlubin/NaNMath.jl.git
INFO: Cloning cache of ReverseDiffSparse from https://github.com/mlubin/ReverseDiffSparse.jl.git
INFO: Installing Calculus v0.2.2
INFO: Installing DiffBase v0.1.0
INFO: Installing ForwardDiff v0.4.2
INFO: Installing JuMP v0.17.1
INFO: Installing MathProgBase v0.6.4
INFO: Installing NaNMath v0.2.6
INFO: Installing ReverseDiffSparse v0.7.3
INFO: Package database updated


In [8]:
Pkg.add("JuMPeR")
Pkg.add("Gurobi")

INFO: Cloning cache of JuMPeR from https://github.com/IainNZ/JuMPeR.jl.git
INFO: Cloning cache of Lazy from https://github.com/MikeInnes/Lazy.jl.git
INFO: Cloning cache of MacroTools from https://github.com/MikeInnes/MacroTools.jl.git
INFO: Downgrading ForwardDiff: v0.4.2 => v0.2.5
INFO: Downgrading JuMP: v0.17.1 => v0.14.2
INFO: Installing JuMPeR v0.5.0
INFO: Installing Lazy v0.11.7
INFO: Installing MacroTools v0.3.7
INFO: Downgrading MathProgBase: v0.6.4 => v0.5.10
INFO: Downgrading ReverseDiffSparse: v0.7.3 => v0.5.8
INFO: Package database updated
INFO: Cloning cache of Gurobi from https://github.com/JuliaOpt/Gurobi.jl.git
INFO: Installing Gurobi v0.3.3
INFO: Building Gurobi
INFO: Package database updated


In [26]:
function solve_stupid_model()

    # Get problem data.
    num_partitions = 2
    jobsStart = [50,233,233]
    jobsEnd = [150,350,350]
    num_jobs = 3
    num_workers = 2
    num_intervals = 2
    cost_backup = 10
    M = 10000

    # Create model.
    m = RobustModel(solver=GurobiSolver(OutputFlag=0))

    ### Static

    # Epigraph variable.
    @variable(m, r)

    # Assignment variables for workers to jobs.
    @variable(m, x[1:num_workers, 1:num_jobs], Bin)

    # Backup agent variables.
    @variable(m, z[1:num_jobs], Bin)

    ## Adjustable

    # Workers to jobs.
    @variable(m, x_u[1:num_partitions, 1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

    # Backup agents.
    @variable(m, z_u[1:num_partitions, 2:num_intervals, 1:num_jobs], Bin)

    # Variables to capture clashes.
    @variable(m, y_u[1:num_partitions, 1:num_jobs, 1:num_jobs], Bin)
 
    ## Uncertainty

    # One uncertain vector for each partition.
    @uncertain(m, u1[1:num_jobs])
    @constraint(m, norm(u1,1) <= 200)
    @constraint(m, norm(u1,Inf) <= 100)
    #apply_constraints!(m, u1, base)
    #apply_constraints!(m, u1, partitions[1])
    @constraint(m, 2*u1[1] - u1[2] <=50)
    
    @uncertain(m, u2[1:num_jobs])
    @constraint(m, norm(u2,1) <= 200)
    @constraint(m, norm(u2,Inf) <= 100)
    #apply_constraints!(m, u2, base)
    #apply_constraints!(m, u2, partitions[2])
    @constraint(m, -2*u1[1] + u1[2] <=-50)


    # Constraint on the tasks not assigned during interval t.
    initial_jobs = [1,2,3]
    @constraint(m, z[initial_jobs] .== 0)
    for i in 1:num_workers

        # Jobs in interval 1 are static.
        @constraint(m, x[i, initial_jobs] .== 0)

        # Remaining jobs are adjustable.
        for t in 2:num_intervals, p in 1:num_partitions
            interval_jobs = [1]
            #@constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
            #@constraint(m, z_u[p, t, :][interval_jobs] .== 0)
        end
    end

    # Objective function.
    @objective(m, Min, r)
    for p in 1:num_partitions
        @constraint(m,  r >= cost_backup * sum(z_u[p, 2, 2] + z_u[p, 2, 3]+z_u[p, 2, 1]))
    end

    # Convering constraints for first stage.
    for j in [1]
        #@constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
    end

    # Covering constraints for adjustable stages.
    for t in 2:num_intervals, p in 1:num_partitions
        for j in [1,2,3]
            @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
        end
    end

    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs
        if j != k
            j_end = jobsEnd[j]
            k_start = jobsStart[k]
            @constraint(m, - M * y_u[1, j, k] <= k_start + u1[k] - (j_end + u1[j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end
    
    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs
        if j != k
            j_end = jobsEnd[j]
            k_start = jobsStart[k]
            @constraint(m, - M * y_u[2, j, k] <= k_start + u2[k] - (j_end + u2[j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end

    # No clash for initial jobs with any other jobs.
    for i in 1:num_workers
        for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_partitions

            # Clashes of initial jobs with other initial jobs.
            @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

            # Clashes of initial jobs with adjustable jobs.
            for t in 2:num_intervals
                @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    # No clash for adjustable jobs.
    for i in 1:num_workers
        for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_partitions
            for j in 1:num_jobs, k in (j + 1):num_jobs
                @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    solve(m)
    @show(getvalue(x_u))
    @show(getvalue(y_u))
    return m
end

solve_stupid_model (generic function with 1 method)

In [27]:
solved = solve_stupid_model()
println(getobjectivevalue(solved))


Academic license - for non-commercial use only
getvalue(x_u) = x_u: 4 dimensions:
[1,:,:,:]
  [1,1,:,:]
    [1,1,2,:]
      [1,1,2,1] = 0.0
      [1,1,2,2] = 0.0
      [1,1,2,3] = 1.0
  [1,2,:,:]
    [1,2,2,:]
      [1,2,2,1] = 1.0
      [1,2,2,2] = 1.0
      [1,2,2,3] = 0.0
[2,:,:,:]
  [2,1,:,:]
    [2,1,2,:]
      [2,1,2,1] = 0.0
      [2,1,2,2] = 1.0
      [2,1,2,3] = 0.0
  [2,2,:,:]
    [2,2,2,:]
      [2,2,2,1] = 0.0
      [2,2,2,2] = 0.0
      [2,2,2,3] = 1.0

getvalue(y_u) = [0.0 1.0 1.0; 0.0 1.0 1.0]

[0.0 0.0 1.0; 1.0 0.0 1.0]

[1.0 1.0 0.0; 1.0 1.0 0.0]
10.0
